In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from herbie import Herbie
from datetime import datetime

 ╭─▌▌Herbie─────────────────────────────────────────────╮
 │ INFO: Created a default config file.                 │
 │ You may view/edit Herbie's configuration here:       │
 │     C:\Users\aaron\.config\herbie\config.toml        │
 ╰──────────────────────────────────────────────────────╯



In [ ]:
class DegreeDayExtractor:
    def __init__(self, weights_path=None):
        """
        weights_path: Path to a CSV/NetCDF containing 'lat', 'lon', and 'weight' 
        (Gas or Pop) for CONUS.
        """
        self.base_temp = 65.0
        # Load weights: If None, we assume uniform weighting for demo
        self.weights = pd.read_csv(weights_path) if weights_path else None

    def get_forecast(self, model='gfs', date=None, fxx=24):
        """
        Models supported by Herbie: 'gfs', 'gefs', 'ifs' (ECMWF), 'aifs' (AI)
        """
        if date is None:
            # Use the most recent 6-hourly cycle, offset by 6h to allow for publication delay
            date = (pd.Timestamp.now('UTC') - pd.Timedelta(hours=6)).floor('6h').strftime("%Y-%m-%d %H:00")

        product = 'pgrb2.0p25' if model in ('gfs', 'gefs') else 'oper'
        H = Herbie(date, model=model, product=product, fxx=fxx)
        ds = H.xarray("TMP:2 m above ground")
        return ds

    def calc_degree_days(self, ds):
        """Calculates HDD and CDD from Kelvin temperature data"""
        # Convert Kelvin to Fahrenheit
        temp_f = (ds.t2m - 273.15) * 9/5 + 32
        
        hdd = np.maximum(0, self.base_temp - temp_f)
        cdd = np.maximum(0, temp_f - self.base_temp)
        
        return hdd, cdd

    def apply_weights(self, dd_array, lat, lon):
        """
        Applies gas/pop weights to the degree day grid.
        Interpolates weight points onto the forecast grid using nearest-neighbor,
        then computes sum(DD * weight) / sum(weight).
        """
        if self.weights is None:
            return float(dd_array.mean())

        # Build an xarray DataArray from the weights CSV
        weight_da = xr.DataArray(
            data=self.weights['weight'].values,
            dims='points',
            coords={
                'lat': ('points', self.weights['lat'].values),
                'lon': ('points', self.weights['lon'].values),
            },
        )

        # Interpolate weights onto the forecast grid using nearest-neighbor
        weight_grid = weight_da.groupby('lat').mean().interp(
            lat=lat, lon=lon, method='nearest',
        ).fillna(0)

        weighted_value = float((dd_array * weight_grid).sum() / weight_grid.sum())
        return weighted_value

### Usage: HDD and CDD by day (7-day forecast)

In [ ]:
extractor = DegreeDayExtractor()

# Fetch 7 days of 6-hourly forecasts (fxx=6,12,...,168)
rows = []
for fxx in range(6, 174, 6):
    ds = extractor.get_forecast(model='gfs', fxx=fxx)
    hdd, cdd = extractor.calc_degree_days(ds)
    valid = pd.Timestamp(ds.valid_time.values)
    rows.append({
        'valid_time': valid,
        'date': valid.strftime('%Y-%m-%d'),
        'HDD': float(hdd.mean()),
        'CDD': float(cdd.mean()),
    })

df = pd.DataFrame(rows)
daily = df.groupby('date')[['HDD', 'CDD']].mean().round(2)
print(daily.to_string())